In [1]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# This is the code link https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb

In [ ]:

# 📌 CONFIGURACIÓN DEL PROYECTO

# Umbral NDWI para detectar agua
NDWI_THRESHOLD = 0.2

# Tamaño del buffer (en metros) para agrupar polígonos cercanos
BUFFER_SIZE = 20

# Área mínima (en m²) de los polígonos a conservar
MIN_POLYGON_AREA = 300

# Porcentaje máximo de nubes permitido
CLOUD_COVER_MAX = 10

# Colección Sentinel a usar
SENTINEL_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'

# Carpeta de exportación en Google Drive
EXPORT_FOLDER = 'Azolla Filiculoides - Geojson Lagunas'

# Activar suavizado individual de polígonos (cierra huecos internos sin unir lagos entre sí)
SMOOTH_POLYGONS = True

SMOOTH_BUFFER_SIZE = 40  # ⬅️ new variable to help close internal holes


<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

# Select Area Interactively

In [2]:

# 📦 Imports
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
from ipyleaflet import WidgetControl
from ipywidgets import HTML

# 🌍 Inicialización Earth Engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# 🗺️ Mapa
Map = geemap.Map(center=[-36.5, -60], zoom=6)
Map.add_basemap('Esri.WorldImagery')
Map.add_draw_control()

# 🎚️ NDWI Slider
ndwi_slider = widgets.FloatSlider(
    value=NDWI_THRESHOLD, min=0.1, max=0.5, step=0.01,
    description='NDWI Threshold:', continuous_update=False
)
display(ndwi_slider)

# 🧠 Estado actual
current_roi = None

# 🧹 Función para limpiar capas previas
def remove_ee_layers(map_obj):
    ee_layers = [lyr for lyr in map_obj.layers if hasattr(lyr, 'ee_object')]
    for lyr in ee_layers:
        map_obj.remove_layer(lyr)

# 🧠 Función principal: detección y exportación de cuerpos de agua
def extract_water_polygons(ndwi_threshold):
    if not current_roi:
        print("⚠️ Seleccioná una región.")
        return

    geom = current_roi

    s2 = ee.ImageCollection(SENTINEL_COLLECTION) \
        .filterBounds(geom) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER_MAX)) \
        .sort('system:time_start', False) \
        .first()

    img_date = s2.date().format('YYYY-MM-dd').getInfo()
    print(f"📅 Imagen Sentinel-2: {img_date}")

    remove_ee_layers(Map)

    vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
    Map.addLayer(s2.clip(geom), vis_params, 'Sentinel-2 RGB')

    date_label = HTML(value=f"<b>Imagen:</b> {img_date}")
    date_control = WidgetControl(widget=date_label, position='topright')
    Map.add_control(date_control)

    # NDWI
    ndwi = s2.normalizedDifference(['B3', 'B8']).rename('NDWI')
    water_mask = ndwi.gt(ndwi_threshold)
    water_mask_int = water_mask.updateMask(water_mask).multiply(1).toInt()

    vectors = water_mask_int.reduceToVectors(
        geometry=geom,
        scale=10,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='water',
        bestEffort=True,
        maxPixels=1e9
    )

    vectors = vectors.map(lambda f: f.set('area', f.geometry().area(1)))
    vectors = vectors.filter(ee.Filter.gt('area', MIN_POLYGON_AREA))

    # 🔁 Suavizado individual de polígonos (cierra huecos internos)
    def smooth_and_fill(f):
        g = f.geometry().buffer(BUFFER_SIZE).buffer(-BUFFER_SIZE)
        return ee.Feature(g).copyProperties(f)
    
        if SMOOTH_POLYGONS:
            vectors = vectors.map(lambda f: ee.Feature(
                f.geometry().buffer(SMOOTH_BUFFER_SIZE).buffer(-SMOOTH_BUFFER_SIZE)
            ).copyProperties(f))


    if SMOOTH_POLYGONS:
        vectors = vectors.map(smooth_and_fill)

    union_fc = vectors

    size = union_fc.size().getInfo()
    if size == 0:
        print("⚠️ No se detectaron polígonos.")
        return

    Map.addLayer(ndwi.clip(geom), {'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')
    Map.addLayer(water_mask_int.clip(geom).selfMask(), {'palette': 'blue'}, 'Water Mask')
    Map.addLayer(union_fc, {'color': 'cyan'}, 'Water Polygons')

    print(f"✅ {size} polígonos generados.")

    # Exportar a Google Drive
    export_name = f'agua_{img_date.replace("-", "")}_ndwi{int(ndwi_threshold*100)}'
    task = ee.batch.Export.table.toDrive(
        collection=union_fc,
        description=export_name,
        folder=EXPORT_FOLDER,
        fileNamePrefix=export_name,
        fileFormat='GeoJSON'
    )
    task.start()
    print(f"🚀 Exportando a Drive como: {export_name}.geojson → /{EXPORT_FOLDER}")

# 🎯 Slider trigger
ndwi_slider.observe(lambda change: extract_water_polygons(ndwi_slider.value), names='value')

# ✏️ Dibujo en el mapa
def on_draw_callback(target, action, geo_json):
    global current_roi
    geom_type = geo_json.get('geometry', {}).get('type')
    if geom_type in ['Polygon', 'Rectangle']:
        current_roi = geemap.geojson_to_ee(geo_json)
        extract_water_polygons(ndwi_slider.value)
    else:
        print("⚠️ Usá solo polígono o rectángulo.")

Map.draw_control.on_draw(on_draw_callback)

# Mostrar el mapa
Map


# POTENCIALES MEJORAS FUTURAS:
# * Hacer que corra por un intervalo de NDWI asi podes agarrar todas las masas de agua detectadas de 0.2-0.3 por ejemplo y despues se mergean todos los poligonos entre ellos y se consigue el mejor resultado posible.

C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


FloatSlider(value=0.2, continuous_update=False, description='NDWI Threshold:', max=0.5, min=0.1, step=0.01)

Map(center=[-36.5, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…